# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

# Preproccess

## read data

In [2]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [3]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [4]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [5]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [6]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [7]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,19054290,20046699,30643955,False,NaN,65,NaN,NaN,NaN,NaN,...,WHITE,NaN,34,NaN,NaN,3,0.0,0,NaN,74.50
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,WHITE,NaN,22,NaN,NaN,2,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,UNABLE TO OBTAIN,NaN,31,NaN,NaN,7,0.0,0,NaN,NaN
0,16171124,20167052,38239449,False,NaN,37,NaN,NaN,NaN,NaN,...,HISPANIC/LATINO - DOMINICAN,NaN,15,NaN,NaN,2,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,NaN,27,NaN,NaN,2,NaN,0,NaN,43.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,WHITE,NaN,30,NaN,NaN,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,NaN,32,NaN,NaN,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,2,0.0,0,NaN,89.05


In [8]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [9]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


# LSTM

### Seperate static and dynamic

In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Concatenate, Masking, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam


def createModel2(timeSteps, timeFeatures, staticFeatures):
    # time series layers
    timeInputLayer = Input(shape=(timeSteps, timeFeatures))
    maskingLayer = Masking(mask_value=0.0)(timeInputLayer)
    # cnnLayer = Conv1D(64, 3, activation="relu")(maskingLayer)
    # poolingLayer = MaxPooling1D(2)(cnnLayer)
    seriesLayer = LSTM(64)(maskingLayer)

    # static layers
    staticInputLayer = Input(shape=(staticFeatures,))
    staticLayer = Dense(32, activation="relu")(staticInputLayer)

    # combine layers
    combined = Concatenate(axis=1)([seriesLayer, staticLayer])
    dense1 = Dense(16, activation="relu")(combined)
    dense2 = Dense(1, activation="sigmoid")(dense1)

    model = Model(inputs=[timeInputLayer, staticInputLayer], outputs=dense2)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )

    return model

In [21]:
%load_ext autoreload
%autoreload 2

from utils.prepare_data import normalizeData, patientsToNumpy
from constants import CATEGORICAL_MEASURES
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping



loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, 
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        timeSeriesOnly=True,
    )

    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
        timeSeriesOnly=True,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
        timeSeriesOnly=True,
    )
    
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    
    
    ################### Static ###################
    staticTrainX = trainPatients.getMeasuresBetween(measureTypes="static")
    staticTestX = testPatients.getMeasuresBetween(measureTypes="static")
    staticValX = valPatients.getMeasuresBetween(measureTypes="static")

    staticTrainX, staticTestX, staticValX = normalizeData(
        staticTrainX, staticTestX, staticValX
    )
    
    staticLen = len(staticTrainX.columns)

    staticTrainX = staticTrainX.to_numpy().astype(np.float32)
    staticTestX = staticTestX.to_numpy().astype(np.float32)
    staticValX = staticValX.to_numpy().astype(np.float32) # type: ignore
    
    staticTrainX = np.nan_to_num(staticTrainX, nan=0)
    staticTestX = np.nan_to_num(staticTestX, nan=0)
    


    ################### labels ###################
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    model = createModel2(npTrainX.shape[1], npTrainX.shape[2], staticLen)

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        [npTrainX, staticTrainX],
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=([npValX, staticValX], np.array(valY)),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate([npTestX, staticTestX], testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - AUC: 0.6233 - accuracy: 0.6375 - loss: 0.6672 - precision: 0.5451 - recall: 0.2775 - val_AUC: 0.7344 - val_accuracy: 0.6839 - val_loss: 0.5957 - val_precision: 0.6119 - val_recall: 0.5395
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.8040 - accuracy: 0.7437 - loss: 0.5516 - precision: 0.6658 - recall: 0.7276 - val_AUC: 0.7548 - val_accuracy: 0.6995 - val_loss: 0.5812 - val_precision: 0.6125 - val_recall: 0.6447
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - AUC: 0.8232 - accuracy: 0.7682 - loss: 0.5181 - precision: 0.6784 - recall: 0.7866 - val_AUC: 0.7729 - val_accuracy: 0.6839 - val_loss: 0.5611 - val_precision: 0.6027 - val_recall: 0.5789
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - AUC: 0.8709 - accuracy: 0.8171 - loss: 0.4717 - precision: 0.7749 - recall: 0.8352 - val_AUC: 0.7966 - val_accuracy: 0.7254 - val_loss: 